In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import resource
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# ------- Define modular methods for the task
def log_max_mem_usage():
    print(
        "Current all-time max memory: {} MB".format(
            resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000
        )
    )

dataset.hdf
sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
train.csv
train.csv.zip



In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df.fillna('zxzxzx zxzxzx', inplace=True) # For id: qid2 174364

test_df = pd.read_csv('../input/test.csv')
test_df.fillna('zxzxzx zxzxzx', inplace=True)

train_df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
log_max_mem_usage()
train_df.shape

Current all-time max memory: 798 MB


(404290, 6)

In [4]:
%%time

dataset_store = pd.HDFStore('../input/dataset.hdf', mode='w')
dataset_store.append('train_df', train_df)
dataset_store.append('test_df', test_df)
dataset_store.close()

CPU times: user 15.6 s, sys: 15.8 s, total: 31.4 s
Wall time: 5min 52s


In [4]:
%%time
dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
dd = dataset_store.select('train_df', where=train_df.index[10:20])
dataset_store.close()
print dd

    id  qid1  qid2                                          question1  \
10  10    21    22  Method to find separation of slits using fresn...   
11  11    23    24        How do I read and find my YouTube comments?   
12  12    25    26               What can make Physics easy to learn?   
13  13    27    28        What was your first sexual experience like?   
14  14    29    30  What are the laws to change your status from a...   
15  15    31    32  What would a Trump presidency mean for current...   
16  16    33    34                       What does manipulation mean?   
17  17    35    36  Why do girls want to be friends with the guy t...   
18  18    37    38  Why are so many Quora users posting questions ...   
19  19    39    40  Which is the best digital marketing institutio...   

                                            question2  is_duplicate  
10  What are some of the things technicians can te...             0  
11             How can I see all my Youtube comments?   

In [5]:
# from joblib import Parallel, delayed

# def score_heuristic_batch(samp_index):
#     dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
#     samp = dataset_store.select('train_df', where=samp_index)
#     dataset_store.close()

#     heuristics_scores = []

#     for row in samp.iterrows():
# #         if row[0] and row[0] % 10000 == 0:
# #             print(row[0])
#         heuristics_scores.append(score_row(row))

#     heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)

#     return heuristics_scores


# def heuristic_score_parallel_interface(t_df):
#     return delayed(score_heuristic_batch)(t_df.index)


# def tfidf_score_parallel_interface(t_df):
#     return delayed(get_tfidf_features)(t_df)


# def parallel_scorer(samp, scorer_interface, batch, num_proc):
#     # Consumes 1.5G for batch=1000 and num_proc=4 for tfidf interface
#     # Use tfidf_features::batch=1000, heuristic_features::batch=20000
#     # scorer_interface::[heuristic_score_parallel_interface, tfidf_score_parallel_interface]
#     # Adjust batch depending on the interface used since the memory is dependent on the batch used.

#     with Parallel(n_jobs=num_proc) as parallel:
#         dataset = []
#         is_break = False
#         i = 0

#         while not is_break:
#             payload = []

#             for j in xrange(num_proc):
#                 t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

#                 if t_df.empty:
#                     is_break = True
#                     continue

#                 payload.append(scorer_interface(t_df))
#             print((i + j) * batch)

#             if payload:
#                 results = parallel(payload)
#                 dataset.extend(results)
#                 i += num_proc

#     return pd.concat(dataset)


# def parallel_get_heuristic_scores(samp, batch=10000, num_proc=4):
#     return parallel_scorer(samp, heuristic_score_parallel_interface, batch, num_proc)


# def parallel_get_tfidf_scores(samp, batch=1000, num_proc=4):
#     return parallel_scorer(samp, tfidf_score_parallel_interface, batch, num_proc)

In [6]:
# %%time
# d = parallel_get_heuristic_scores(train_df.head(200000), batch=10000, num_proc=4)
# d

In [7]:
# d

In [8]:
log_max_mem_usage()

Current all-time max memory: 798 MB


In [9]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

unique_questions = pd.Series(pd.concat([train_df.question1, train_df.question2]).unique())
# combined_featurizers.fit(unique_questions)

char_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))  # featurizers[0][1]
word_tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))  # featurizers[1][1]
char_tfidf.fit(unique_questions)
word_tfidf.fit(unique_questions)

log_max_mem_usage()

Current all-time max memory: 2115 MB
CPU times: user 55.3 s, sys: 564 ms, total: 55.9 s
Wall time: 55.6 s


In [10]:
# %%time
# d = get_tfidf_features(train_df[:100000], batch=1000)

In [11]:
from nltk.corpus import stopwords
from collections import Counter
import re


stops = set(stopwords.words("english"))
num_pattern = re.compile('[0-9]+')
math_pattern = re.compile('\[math\](.*)\[\/math\]')

nums = '01234567890'


def get_tfidf_features(data_df, batch=1000):
    # data_df size should be about 40000 when used in parallel to observe effects of optimization.
    i = 0
#     print('Current data size in `get_tfidf_features`: {}'.format(data_df.shape[0]))

    word_dataset = np.array([])
    char_dataset = np.array([])

    while True:
        samp = data_df[i * batch: (i + 1) * batch]
        i += 1
#         print_batch = i * batch % 10000

#         if print_batch == 0:
#             print('Batch process in `get_tfidf_features`: {}'.format(i * batch))

        if samp.empty:
            break

        word_res = np.dot(
            word_tfidf.transform(samp.question1),
            word_tfidf.transform(samp.question2).T
        ).diagonal()
        
        char_res = np.dot(
            char_tfidf.transform(samp.question1),
            char_tfidf.transform(samp.question2).T
        ).diagonal()

        word_dataset = np.concatenate([word_dataset, word_res])
        char_dataset = np.concatenate([char_dataset, char_res])

    return pd.DataFrame(dict(wv=word_dataset, cv=char_dataset), index=data_df.index)


def get_heuristic_scores(q1, q2, ns_q1, ns_q2, swap):
#     n_q1 = {}
#     n_q2 = {}

#     for n in nums:
#         qc1 = q1.count(n)
#         qc2 = q2.count(n)
#         n_q1['q1_{}'.format(n)] = qc1
#         n_q2['q2_{}'.format(n)] = qc2

    if swap:
        q1 = ns_q1
        q2 = ns_q2

    exact_nums_q1 = num_pattern.findall(q1)
    exact_nums_q2 = num_pattern.findall(q2)
    
    math_q1 = math_pattern.findall(q1)
    math_q2 = math_pattern.findall(q2)

    num_exact_nums_match = len([n1 for n1 in exact_nums_q1 if n1 in exact_nums_q2])
    math_pattern_match = len([n1 for n1 in math_q1 if n1 in math_q2])
    
    is_q1_math = 1 * any(math_q1)
    is_q2_math = 1 * any(math_q2)
    is_both_math = is_q1_math * is_q2_math

#     qq2 = pd.Series(Counter([s for s in q1 if s.isupper()]))
#     qq1 = pd.Series(Counter([s for s in q2 if s.isupper()]))
    
#     sim_caps_rate = (qq1/qq2).mean()
#     num_caps_q1 = qq1.sum() 
#     num_caps_q2 = qq2.sum()

#     mean_caps_q1 = qq1.mean() 
#     mean_caps_q2 = qq2.mean()
    
    num_terms_q1 = len(q1.split())
    num_terms_q2 = len(q2.split())
    
    len_q1 = len(q1)
    len_q2 = len(q2)

    res = dict(
        num_exact_nums_match=num_exact_nums_match,
        math_pattern_match=math_pattern_match,
        is_q1_math=is_q1_math,
        is_q2_math=is_q2_math,
        is_both_math=is_both_math,
        length_diff=abs(len_q1 - len_q2),
        len_q1=len_q1,
        len_q2=len_q2,
        word_num_diff=abs(num_terms_q1 - num_terms_q2),
        num_terms_q1=num_terms_q1,
        num_terms_q2=num_terms_q2,
#         sim_caps_rate=sim_caps_rate,
#         mean_caps_q1=mean_caps_q1,
#         mean_caps_q2=mean_caps_q2,
#         num_caps_q1=num_caps_q1,
#         num_caps_q2=num_caps_q2,
    )
    
    # res.update(n_q1)
    # res.update(n_q2)
    
    return res


heuristics_feature_names = [
    'num_exact_nums_match',
    'math_pattern_match',
    'is_q1_math',
    'is_q2_math',
    'is_both_math',
    'length_diff',
    'len_q1',
    'len_q2',
    'word_num_diff',
    'num_terms_q1',
    'num_terms_q2',
]


def score_row(row, check_stops=False, swap=False):
    ix, row = row

    q1 = row.question1
    q2 = row.question2
    
    ns_q1 = [i for i in q1.lower().split() if i not in stops]
    ns_q2 = [i for i in q1.lower().split() if i not in stops]

    if not all([ns_q1, ns_q2]) and check_stops:
        if ix % 5 == 0:
            # 39405
            print('here! {}'.format(ix))

        return {i: np.nan for i in heuristics_feature_names}
    
    ns_q1 = ' '.join(ns_q1)
    ns_q2 = ' '.join(ns_q2)

    return get_heuristic_scores(q1, q2, ns_q1, ns_q2, swap=swap)

In [12]:
ix =  203
row = train_df.ix[ix]
q1 = row.question1
q2 = row.question2
print q1
print q2
print set(q1.lower().split()).difference(stops)
print set(q2.lower().split()).difference(stops)
print row.is_duplicate
print score_row((ix, row))

Why do people hate Hillary Clinton?
What are the reasons that people dislike Hillary Clinton?
set(['hillary', 'hate', 'clinton?', 'people'])
set(['reasons', 'hillary', 'dislike', 'clinton?', 'people'])
1
{'is_q2_math': 0, 'math_pattern_match': 0, 'is_q1_math': 0, 'length_diff': 22, 'num_terms_q1': 6, 'num_terms_q2': 9, 'is_both_math': 0, 'num_exact_nums_match': 0, 'word_num_diff': 3, 'len_q1': 35, 'len_q2': 57}


In [13]:
# %%time
# ds = []
# # samp = train_df[404000:]
# samp = train_df.head(200000)  # [30000:50000]
# for row in samp.iterrows():
#     ds.append(score_row(row))

# ds = pd.DataFrame(ds, index=samp.index)
# #ds

In [15]:
# %%time
from sklearn.model_selection import train_test_split
import multiprocessing as mp
import time
from joblib import Parallel, delayed


def score_heuristic_batch(samp_index, is_train):
    dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
    if is_train:
        samp = dataset_store.select('train_df', where=samp_index)
    else:
        samp = dataset_store.select('test_df', where=samp_index)
    dataset_store.close()

    heuristics_scores = []

    for row in samp.iterrows():
#         if row[0] and row[0] % 10000 == 0:
#             print(row[0])

        heuristics_scores.append(score_row(row))

    heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)

    return heuristics_scores


def heuristic_score_parallel_interface(t_df, is_train):
    return delayed(score_heuristic_batch)(t_df.index, is_train)


def tfidf_score_parallel_interface(t_df, is_train):
    return delayed(get_tfidf_features)(t_df)


def parallel_scorer(samp, scorer_interface, is_train, batch, num_proc):
    # Consumes 1.5G for batch=1000 and num_proc=4 for tfidf interface
    # Use tfidf_features::batch=10000, heuristic_features::batch=20000
    # scorer_interface::[heuristic_score_parallel_interface, tfidf_score_parallel_interface]
    # Adjust batch depending on the interface used since the memory is dependent on the batch used.

    with Parallel(n_jobs=num_proc) as parallel:
        dataset = []
        is_break = False
        i = 0

        while not is_break:
            payload = []

            for j in xrange(num_proc):
                t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

                if t_df.empty:
                    is_break = True
                    continue

                payload.append(scorer_interface(t_df, is_train))
            print('Current batch in main thread: {}'.format((i + j) * batch))

            if payload:
                results = parallel(payload)
                dataset.extend(results)
                i += num_proc

    return pd.concat(dataset)


def parallel_get_heuristic_scores(samp, is_train, batch=10000, num_proc=4):
    return parallel_scorer(samp, heuristic_score_parallel_interface, is_train, batch, num_proc)


def parallel_get_tfidf_scores(samp, is_train, batch=40000, num_proc=4):
    # The batch size for the size of the dataset should be large to maximize effect of parallelization.
    # The batch here is different from the batch used in the method `get_tfidf_features`
    return parallel_scorer(samp, tfidf_score_parallel_interface, is_train, batch, num_proc)


NUM_PROC = max(1, mp.cpu_count() - 1)

def parallel_get_features(samp, is_train, tfidf_sample_batch=40000, heuristics_sample_batch=10000):
    start_time = time.time()

    tfidf_features = parallel_get_tfidf_scores(samp, is_train, batch=tfidf_sample_batch, num_proc=NUM_PROC)
    print('Finished computing tfidf features after {} seconds.'.format((time.time() - start_time)))

    heuristics_scores = parallel_get_heuristic_scores(samp, is_train, batch=heuristics_sample_batch, num_proc=NUM_PROC)
    print('Finished computing heuristic features after {} seconds.'.format((time.time() - start_time)))

    features = pd.concat([tfidf_features, heuristics_scores], axis=1)
    return features


def get_features(samp):
    start_time = time.time()
    
    tfidf_features = get_tfidf_features(samp)
    print('Finished computing tfidf features after {} seconds.'.format((time.time() - start_time)))

    heuristics_scores = []

    for row in samp.iterrows():
        if row[0] and row[0] % 10000 == 0:
            print(row[0])
  
        heuristics_scores.append(score_row(row))

    heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)
    
    features = pd.concat([tfidf_features, heuristics_scores], axis=1)
    
    return features


# samp = train_df

# X_train = get_features(samp)
# y_train = samp.is_duplicate

# log_max_mem_usage()

In [104]:
import psutil
GB = 1000000000.0  # Not really GB :D
SAVE_GB = 1.5 * GB

def get_tfidf_batch_size(df, NUM_PROC):
    # Memory (4.2 - 2.01 = 2.19G) used with 7 processes and 404290 rows (train) with batch size 57755

    mem_per_row = 2.19 / 57755 / 7
    available_mem = (psutil.virtual_memory().available - SAVE_GB) / GB
    tfidf_batch_size = df.shape[0] / NUM_PROC
    delta = 10000

    if (df.shape[0] * mem_per_row) > available_mem:
        while (tfidf_batch_size * NUM_PROC * mem_per_row) > available_mem:
            tfidf_batch_size -= delta

    return tfidf_batch_size

NUM_PROC=7
train_tfidf_batch_size = get_tfidf_batch_size(train_df, NUM_PROC=NUM_PROC)
train_tfidf_batch_size

57755

In [105]:
%%time
dtfidf = parallel_get_tfidf_scores(train_df, is_train=True, batch=train_tfidf_batch_size, num_proc=NUM_PROC)
# get_tfidf_features(train_df.head(60000), batch=1000)  # , num_proc=NUM_PROC)

Current batch in main thread: 346530
Current batch in main thread: 750815
CPU times: user 1.05 s, sys: 200 ms, total: 1.25 s
Wall time: 42.1 s


In [106]:
%%time
dheur = parallel_get_heuristic_scores(train_df, is_train=True, batch=train_tfidf_batch_size, num_proc=NUM_PROC)

Current batch in main thread: 346530
Current batch in main thread: 750815
CPU times: user 132 ms, sys: 200 ms, total: 332 ms
Wall time: 12.7 s


In [88]:
# heuristics_scores = []
# samp = train_df[404285: 404293]
# for row in samp.iterrows():
#     if row[0] and row[0] % 10000 == 0:
#         print(row[0])

#     heuristics_scores.append(score_row(row))
# heuristics_scores

In [43]:
ix = 410000
batch = 30000
samp = test_df[ix: ix + batch]
print(samp.index)
dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
# sp = dataset_store.select('train_df', where=samp.index)
sp = dataset_store.select('test_df', where=samp.index)
dataset_store.close()
# sp

RangeIndex(start=410000, stop=440000, step=1)


In [107]:
%%time
train_batch_size = get_tfidf_batch_size(train_df, NUM_PROC=NUM_PROC)

X_train = parallel_get_features(
    train_df, is_train=True, tfidf_sample_batch=train_batch_size, heuristics_sample_batch=train_batch_size
)
y_train = train_df.is_duplicate

log_max_mem_usage()

Current batch in main thread: 346530
Current batch in main thread: 750815
Finished computing tfidf features after 41.7312591076 seconds.
Current batch in main thread: 346530
Current batch in main thread: 750815
Finished computing heuristic features after 54.2434220314 seconds.
Current all-time max memory: 2115 MB
CPU times: user 1.04 s, sys: 376 ms, total: 1.42 s
Wall time: 54.3 s


In [30]:
# %%time
# X_train = get_features(samp)
# y_train = samp.is_duplicate

# log_max_mem_usage()

In [31]:
train_df.shape

(404290, 6)

In [131]:
train_sample_dist = train_df.groupby('is_duplicate').count()['id']

p = 0.165
a = train_sample_dist[1]
b = train_sample_dist[0]

neg_extra = int((a / p) - b)
neg_extra_ind = np.random.choice(train_df[train_df.is_duplicate == 0].index, size=neg_extra, replace=True)

os_X_train = pd.concat([X_train, X_train.ix[neg_extra_ind]])
os_y_train = pd.concat([y_train, y_train.ix[neg_extra_ind]])

os_X_train, X_valid, os_y_train, y_valid = train_test_split(os_X_train, os_y_train, test_size=0.2, random_state=1029)

log_max_mem_usage()

Current all-time max memory: 2115 MB


In [138]:
y_valid.shape

(210778,)

In [132]:
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, make_scorer


scaler = StandardScaler()

char_lm_model = LogisticRegression(C=100)
word_lm_model = LogisticRegression(C=1)
length_diff_lm_model = LogisticRegression(C=1)
word_num_diff_lm_model = LogisticRegression(C=1)

rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, min_samples_split=3, n_jobs=NUM_PROC)
lm_model = LogisticRegression()

def log_loss_scorer(model, X, y):
    return log_loss(y, model.predict_proba(X))

def fit_models(X, y):
    rf_model.fit(X_train, y_train)
    char_lm_model.fit(X_train.cv.values.reshape(-1, 1), y_train)
    word_lm_model.fit(X_train.wv.values.reshape(-1, 1), y_train)
    length_diff_lm_model.fit(X_train.length_diff.values.reshape(-1, 1), y_train)
    word_num_diff_lm_model.fit(X_train.word_num_diff.values.reshape(-1, 1), y_train)

def predict(X):
#     weights = dict(zip(X.columns, rf_model.feature_importances_))
    char_pred = char_lm_model.predict_proba(X.cv.values.reshape(-1, 1))[:, 1] # * weights['cv']
    word_pred = word_lm_model.predict_proba(X.wv.values.reshape(-1, 1))[:, 1] # * weights['wv']
    length_diff_pred = length_diff_lm_model.predict_proba(X.length_diff.values.reshape(-1, 1))[:, 1] # * weights['length_diff']
    word_num_diff_pred = word_num_diff_lm_model.predict_proba(X.word_num_diff.values.reshape(-1, 1))[:, 1] # * weights['word_num_diff']
    rf_pred = rf_model.predict_proba(X)[:, 1]

    return [char_pred, word_pred, length_diff_pred, word_num_diff_pred, rf_pred]

In [133]:
%%time

lm_model.fit(os_X_train, os_y_train)

print(log_loss_scorer(lm_model, X_valid, y_valid))

0.344890944025


In [134]:
%%time

rf_model.fit(os_X_train, os_y_train)

print(log_loss_scorer(rf_model, X_valid, y_valid))

0.211987538739


In [136]:
%%time
# fit_models(X_train, y_train)
cvs = cross_val_score(rf_model, os_X_train, os_y_train, scoring=log_loss_scorer)
print(cvs)

[ 0.24221531  0.24187433  0.24231307]
CPU times: user 11min 7s, sys: 1.11 s, total: 11min 9s
Wall time: 1min 40s


In [139]:
print(roc_auc_score(os_y_train, rf_model.predict_proba(os_X_train)[:, 1]))
print(roc_auc_score(y_valid, rf_model.predict_proba(X_valid)[:, 1]))

0.999755690807
0.954475840628


In [15]:
# print(log_loss(y_test, np.mean(predict(X_test), axis=0)))
# print(roc_auc_score(y_test, np.mean(predict(X_test), axis=0)))
# print(roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))
# print(sum((y_test - (1 * (rf_model.predict_proba(X_test)[:, 1] < 0.5))) != 0))

In [16]:
# for i, j in sorted(list(zip(X_test.columns, rf_model.feature_importances_)), key=lambda x: x[1], reverse=True):
#     print i, j

In [17]:
# pd.concat([features, samp.is_duplicate], axis=1).groupby('is_duplicate').mean().T

In [23]:
test_df.fillna('zxzxzx zxzxzx', inplace=True)

In [140]:
1.0 * test_df.shape[0] / train_df.shape[0]

5.802260753419575

In [141]:
%%time

test_batch_size = get_tfidf_batch_size(test_df, NUM_PROC=NUM_PROC)
print('Tes batch size: {}'.format(test_batch_size))

X_test = parallel_get_features(
    test_df, is_train=False, tfidf_sample_batch=test_batch_size, heuristics_sample_batch=test_batch_size
)

log_max_mem_usage()

Tes batch size: 255113
Current batch in main thread: 1530678
Current batch in main thread: 3316469
Finished computing tfidf features after 263.97646594 seconds.
Current batch in main thread: 1530678
Current batch in main thread: 3316469
Finished computing heuristic features after 356.002717018 seconds.
Current all-time max memory: 4982 MB
CPU times: user 4.98 s, sys: 1.1 s, total: 6.08 s
Wall time: 5min 56s


In [142]:
%%time
X_test.to_hdf('kaggle-quora', 'test-features-X_test')

/home/avsolatorio/anaconda/lib/python2.7/site-packages/tables/path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'test-features-X_test'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


CPU times: user 128 ms, sys: 164 ms, total: 292 ms
Wall time: 293 ms


In [143]:
test_id = test_df.test_id

# del(test_df)
# del(train_df)

In [144]:
import gc
gc.collect()

75

In [145]:
from datetime import datetime

In [146]:
# %%time
lm_sub = pd.DataFrame()

lm_sub['test_id'] = test_id
lm_sub['is_duplicate'] = lm_model.predict_proba(X_test)[:, 1]

lm_sub.to_csv('lm_submission_{}.csv'.format(datetime.now()), index=False)
lm_sub.head()

,test_id,is_duplicate
0,0,0.046317
1,1,0.119348
2,2,0.160049
3,3,0.030699
4,4,0.027189


In [50]:
# # %%time
# rf_sub = pd.DataFrame()

# rf_sub['test_id'] = test_id
# rf_sub['is_duplicate'] = rf_model.predict_proba(X_test)[:, 1]

# rf_sub.to_csv('rf_submission.csv', index=False)
# rf_sub.head()

,test_id,is_duplicate
0,0,0.048285
1,1,0.365262
2,2,0.149548
3,3,0.005025
4,4,0.173174


In [147]:
# %%time
rf_sub = pd.DataFrame()

rf_sub['test_id'] = test_id
rf_sub['is_duplicate'] = rf_model.predict_proba(X_test)[:, 1]

rf_sub.to_csv('rf_submission_{}.csv'.format(datetime.now()), index=False)
rf_sub.head()

,test_id,is_duplicate
0,0,0.026000
1,1,0.302941
2,2,0.198157
3,3,0.024167
4,4,0.132272


In [155]:
test_df[rf_sub.is_duplicate > 0.4].head()

,test_id,question1,question2
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...
8,8,What are the how best books of all time?,What are some of the military history books of...
10,10,What is the best slideshow app for Android?,What are the best app for android?
17,17,Can a vacuum cleaner concentrate suck your eye...,Could a vacuum cleaner suck get your eye out i...
35,35,What site the best example of dedication in an...,What are some of the best examples of new in a...


In [157]:
for r in test_df[rf_sub.is_duplicate > 0.4].head().iterrows():
    ix, r = r
    print(rf_sub.ix[ix].is_duplicate)
    print(r.question1)
    print(r.question2)
    print()

0.684519662921
By scrapping the 500 and 1000 rupee notes, how is RBI planning to fight against issue black money?
How will the recent move to declare 500 and 1000 denomination lewin illegal will curb black money?
()
0.48367564957
What are the how best books of all time?
What are some of the military history books of all time?
()
0.570220336466
What is the best slideshow app for Android?
What are the best app for android?
()
0.755877344877
Can a vacuum cleaner concentrate suck your eye out if it is pressed against your face?
Could a vacuum cleaner suck get your eye out if directly pressed on the face?
()
0.587628221401
What site the best example of dedication in any field?
What are some of the best examples of new in any field?
()


In [ ]:
from IPython.display import FileLink
test_df = pd.read_csv('../input/test.csv')

scores_data = []
samp = train_df

for row in samp.iterrows():
    if row[0] % 5000 == 0:
        print(row[0])

    scores_data.append(score_row(row))

X = pd.DataFrame(scores_data)
is_duplicate_test = np.mean(predict(X), axis=0)

log_max_mem_usage()

In [ ]:
del(train_df)